<img src="img/swarm-logo.png">
https://hub.docker.com/_/swarm

Docker Swarm и его младший брат Docker Compose - это системы для управления сразу целым набором контейнеров, которые могут коммуницировать между собой и представлять для пользователя единый интерфейс для взаимодействия.

Как и в предыдущей лабораторной, мы будем использовать сервис Play With Docker для демонстрации возможностей этого инструмента.

# [Открыть Play With Docker](https://labs.play-with-docker.com/)

Для начала создадим одну виртуальную машину.

# Docker-compose

Docker compose конфигурация - это YAML файл, в котором указаны параметры запуска контейнеров.

Официальную документацию по формату можно найти по следующей ссылке - https://docs.docker.com/compose/compose-file/

Попробуем для начала создать конфигурацию для того сервиса, который мы собрали в предыдущей лабороторной.

Для начала создадим точно такой же Dockerfile.

**Dockerfile**
```dockerfile
FROM python:3.7

RUN mkdir /sync-folder

WORKDIR /sync-folder

ENTRYPOINT ["python3", "-m", "http.server", "--bind", "0.0.0.0", "8080"]
```

Теперь создадим docker-compose.yaml и укажем в нем один этот сервис.

**docker-compose.yaml**
```yaml
version: "3.7"

services:
  file-server:
    build:
      context: .  # Указываем директорию, где лежит Dockerfile
    ports:
      - 9090:8080  # Указываем, какие порты пробросить
    volumes:
      - ./:/sync-folder  # Указываем, какие директории примонтировать
```

Сохраним оба файла, можно запускать

```bash
docker-compose up --build
```

После запуска этой команды, контейнер должен собраться и запуститься. Проверить работоспособность можно открыв порт 9090 кнопкой "OPEN PORT".

# Пробрасываем конфигурацию

Помимо монтирования целой директории, docker позволяет подключать единичные файлы. Это полезно, например, для подключения файла конфигурации внутри контейнеризированного приложения. Дополнительно можно выставить флаг Read-Only, чтобы обезопасить оригинальный файл с конфигурацией от изменений.

Попробуем написать скрипт, который читает из импровизированной конфигурации путь до директории и именно из нее начинает раздавать файлы.

**file-server.sh**
```bash
export DIR=$(cat /etc/file-config.txt)

mkdir -p $DIR

python3 -m http.server --directory "$DIR" --bind 0.0.0.0 8080
```

**Dockerfile**
```dockerfile
FROM python:3.7

COPY file-server.sh file-server.sh

ENTRYPOINT ["bash", "file-server.sh"]
```

Теперь создаем сам файл конфигурации

**config.txt**
```
/shared/folder
```

И подключим его к сервису

**docker-compose.yaml**
```yaml
version: "3.7"

services:
  file-server:
    build:
      context: .  # Указываем директорию, где лежит Dockerfile
    ports:
      - 9090:8080  # Указываем, какие порты пробросить
    volumes:
      - ./:/shared/folder  # Указываем, какие директории примонтировать
      - ./config.txt:/etc/file-config.txt:ro
```

Запускаем написанный нами compose
```bash
docker-compose up --build
```

И проверяем работу на `9090`.

# Взаимодействие контейнеров между собой

Для удобства разработки, Docker Compose создает виртуальную сеть между всеми контейнерами. В этой виртуальной сети каждый контейнер может сделать запрос к любому другому просто по имени этого контейнера. Например, таким образом можно соединять контейнеризированные приложения с контейнеризированными базами данных.

Мы сейчас попробуем создать два контейнера - один будет нашим файловым сервером, второй будет прокси-сервером. То есть мы будем как пользователи работать только с прокси-сервером, а он будет перенаправлять наши запросы к нужному настоящему приложению.

В качестве прокси-сервера будем использовать Nginx. Это самый популярный web-сервер, для которого уже есть готовый контейнер. Чтобы он работал корректно, необходимо лишь передать ему корректную конфигурацию.

Создадим вначале эту самую конфигурацию. Она пишется в специальном формате. Более подробно про него почитать можно на официальном сайте - https://nginx.org/ru/docs/beginners_guide.html#conf_structure

**configuration.nginx**
```
user root;
worker_processes  4;

error_log  /var/log/nginx/error.log warn;
pid        /var/run/nginx.pid;

events {
    worker_connections  4096;
}

http {
    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;
    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile            on;
    tcp_nopush          on;
    keepalive_timeout   65;

    server {
        listen  8888;

        location /file-server-1 {
            proxy_pass http://file-server:8080;   # Перенаправляем в соседний контейнер по именно контейнера!
            rewrite ^/file-server-1/(.*) /$1 break;  # Удаляем префикс /web-server-1
        }
    }
}
```

**docker-compose.yaml**
```yaml
version: "3.7"

services:
  file-server:
    build:
      context: .
    volumes:
      - ./:/shared/folder
      - ./config.txt:/etc/file-config.txt:ro
  proxy:
    image: nginx:1.17
    volumes:
      - ./configuration.nginx:/etc/nginx/nginx.conf:ro
    ports:
      - 8888:8888
```

Пробуем запустить
```bash
docker-compose up --build
```

Важно заметить, что `file-server` больше не пробрасывает порты. А прокси-сервер работает на `8888` и слушает запросы по пути `/file-server-1/*`.

Таким образом после запуска, необходимо открыть `8888` через интерфейс и в адресной строке добавить `/file-server-1/`.

Пример того, какая адресная строка должна получиться - `http://ip172-18-0-10-bspo3glim9m000eq1ltg-8888.direct.labs.play-with-docker.com/file-server-1/`.

# Сразу три контейнера

Чтобы эффект от того, что мы сделали, ощущался более явно, давайте добавим еще один файловый сервер и подключим к нему наш прокси.


**docker-compose.yaml**
```yaml
version: "3.7"

services:
  file-server-1:
    build:
      context: .
    volumes:
      - ./:/shared/folder # Текущая директория
      - ./config.txt:/etc/file-config.txt:ro 
  file-server-2:
    build:
      context: .
    volumes:
      - ./config.txt:/etc/file-config.txt:ro  
      - /etc:/shared/folder # Директория /etc
  proxy:
    image: nginx:1.17
    volumes:
      - ./configuration.nginx:/etc/nginx/nginx.conf:ro
    ports:
      - 8888:8888
```

И обновим конфигурацию

**configuration.nginx**
```
user root;
worker_processes  4;

error_log  /var/log/nginx/error.log warn;
pid        /var/run/nginx.pid;

events {
    worker_connections  4096;
}

http {
    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;
    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile            on;
    tcp_nopush          on;
    keepalive_timeout   65;

    server {
        listen  8888;

        location /file-server-1 {
            proxy_pass http://file-server-1:8080;   # Перенаправляем в соседний контейнер по именно контейнера!
            rewrite ^/file-server-1/(.*) /$1 break;  # Удаляем префикс /web-server-1
        }
        
        location /file-server-2 {
            proxy_pass http://file-server-2:8080;
            rewrite ^/file-server-2/(.*) /$1 break;
        }
    }
}
```

Собираем, запускаем и вновь смотрим на 8888 порт.
```bash
docker-compose up --build
```

Ожидаемое поведение следующее 
* Если мы откроем /file-server-1/ то мы увидим содержимое текущей директории основной ОС
* Если мы откроем /file-server-2/ то мы увидим содержимое директории /etc основной ОС

Для наглядности вот схема того, как запущенная нами система работает.

<img src="img/proxy-with-file-server-schema.png">